# Генератор (болталка)

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 50.8 MB/s 
     |████████████████████████████████| 182 kB 55.2 MB/s 


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
#DEVICE = torch.device("cuda:0")

model_name_or_path = "radm/rugpt3medium-tathagata"
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)#.to(DEVICE)

In [4]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=end_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

#"Бот-всезнайка"

In [5]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=52c266a1defe786d314eecb4217a1f8723ca4df72a6fc8bef327af10efa5d17f
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [10]:
import wikipedia
import re
wikipedia.set_lang("ru")

In [11]:
# Чистим текст статьи из Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:2000]
        # Разбиваем, считая точку разделителем
        wikimas=wikitext.split('.')
        # Отбрасываем всё после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
                    # Если в строке осталось больше трех символов, добавляем ее
                    # к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'В энциклопедии нет информации об этом'

In [12]:
print(getwiki('Минск'))

Минск  — столица и крупнейший город Белоруссии, административный центр Минской области и Минского района, в состав которых не входит, поскольку является самостоятельной административно-территориальной единицей с особым  статусом. Крупнейший транспортный узел, политический, экономический, культурный и научный центр страны. Является ядром Минской агломерации. Десятый по численности населения  город в Европе, пятый — после Москвы, Санкт-Петербурга, Киева, Ташкента на территории бывшего СССР. Город расположен недалеко от географического центра страны и стоит на реке Свислочи. Площадь составляет 348,84 км², население — 1 996 553 человек без учёта пригородов. Город-герой .


# Bot

In [13]:
!pip install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
history_dialog = []

def babbler(text):
    global history_dialog

    history_dialog.append(text)
    history_dialog.append(respond_to_dialog(history_dialog[-10:]))
    if len(history_dialog) > 10: history_dialog = history_dialog[-10:]
    
    return history_dialog[-1]

In [38]:
import telebot
from telebot import types

# Создаем экземпляр бота
bot = telebot.TeleBot("5943099843:AAGU5zDXO7ftv7sQpJ5TD7EjpvOu-So8g6w")

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Привет")
    markup.add(btn1)
    bot.send_message(message.chat.id, "👋 Привет! Я твой бот-помошник!", reply_markup=markup)

Flag = 0
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    global Flag
    # Если нажали на одну из кнопок
    if call.data.lower() == 'поболтаем': 
        Flag = 1
        bot.send_message(call.message.chat.id, 'О чём ты хочешь побалтать?')
    if call.data.lower() == 'бот-всезнайка': 
        Flag = 2
        bot.send_message(call.message.chat.id, 'О чём тебе расказать?')


# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    global Flag
    
    if message.text.lower() == 'привет': Flag = 0
    

    if Flag == 0:# or message.text == 'Назад':
        # Готовим кнопки
        keyboard = types.InlineKeyboardMarkup()
        # И добавляем кнопку на экран
        key_oven = types.InlineKeyboardButton(text='Поболтаем', callback_data='поболтаем')
        # И добавляем кнопку на экран
        keyboard.add(key_oven)
        key_telec = types.InlineKeyboardButton(text='Бот-всезнайка', callback_data='бот-всезнайка')
        keyboard.add(key_telec)
        # Показываем все кнопки сразу и пишем сообщение о выборе
        bot.send_message(message.chat.id, text='Сделайте выбор', reply_markup=keyboard)
    
    elif Flag == 1:
        bot.send_message(message.chat.id, babbler(message.text))
    
    elif Flag == 2:
        bot.send_message(message.chat.id, getwiki(message.text))


# Запускаем бота
bot.polling(none_stop=True, interval=0)